In [2]:
# lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
# spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
# city = lo_polygon.union(spb_polygon).to_crs(epsg=4326) #  get lo polygon
# polygon = gpd.GeoDataFrame([{'name': 'Ленинградская область', 'geometry': city[0]}], crs="EPSG:4326")

In [3]:
# gg= he.convert_list_attr_to_str(g.graph)
# for node,e,data in gg.edges(data=True):
#     if 'geometry' in data:
#         data['geometry']=str(data['geometry'])
# nx.write_graphml(gg, f"tula.graphml")

In [2]:
import sys
# sys.path.insert(0, os.path.abspath('..'))
# import os
# folder = os.getcwd().split("\\notebooks")[0]
sys.path.append('..')

from transport_frames.graphbuilder.graph import Graph 
from transport_frames.frame_grader.advanced_grade import AdvancedGrader
from transport_frames.framebuilder.frame import Frame
import transport_frames.utils.helper_funcs as he
from transport_frames.indicators.indicator_terr import indicator_territory
from transport_frames.indicators.indicator_area import indicator_area
from transport_frames.indicators.utils import availability_matrix, create_service_dict
import osmnx as ox
import geonetworkx as gnx
import geopandas as gpd
import networkx as nx
from shapely import Polygon, MultiPolygon
import momepy
import shapely
from dongraphio import GraphType
from iduedu import get_adj_matrix_gdf_to_gdf
from iduedu import get_intermodal_graph



/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Graph creation

In [5]:
geocode=81993
polygon = ox.geocode_to_gdf(f'R{geocode}',by_osmid=True)
local_crs = 32637

In [7]:
# g = Graph.from_polygon(polygon, crs=local_crs)

tula = he.convert_geometry_from_wkt(nx.read_graphml('/Users/polina/Desktop/github/transport_frames/src/tula.graphml'))
tula = he.convert_list_attr_from_str(tula)
g =  Graph(tula,crs=local_crs)

2024-10-04 15:11:19.406 | INFO     | src.graphbuilder.graph:_prepare_attrs:138 - Preparing the graph...
2024-10-04 15:11:28.972 | INFO     | src.graphbuilder.graph:_prepare_attrs:176 - Graph is ready!


## Frame creation

In [8]:
# polygon regions cities
regions = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/regions_of_russia.geojson')
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions[regions['geometry'].apply(lambda x: isinstance(x, (Polygon, MultiPolygon)))]

admin_centers = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
admin_centers = admin_centers[admin_centers.is_admin_centre_district==1][['name','geometry']].reset_index()


In [9]:
f = Frame(g.graph, regions, polygon, admin_centers)
# f.get_geopackage();

KeyboardInterrupt: 

## Grading custom territory (frame-based)

In [11]:
custom_terr = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson').iloc[[5]]

In [12]:
graded_terr = f.grade_territory(custom_terr, include_priority=True)

## Criteria of the territory


In [13]:
settlement_points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
settlement_polygons = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')
railway_stations = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
bus_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
aero = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')

In [ ]:
restricted_terr = [(ox.geocode_to_gdf('Estonia'), 0.0),
                   (ox.geocode_to_gdf('Finland'), 0.5)]

In [14]:
inter = get_intermodal_graph(osm_id=geocode,
                        keep_routes_geom=True,clip_by_bounds=True)

2024-09-23 16:40:14.669 | DEBUG    | iduedu.modules.downloaders:get_boundary_by_osm_id:33 - Downloading territory bounds with osm_id <81993> ...
2024-09-23 16:40:15.061 | DEBUG    | iduedu.modules.drive_walk_builder:get_walk_graph:214 - Downloading walk graph from OSM ...
2024-09-23 16:40:15.061 | DEBUG    | iduedu.modules.intermodal_builder:get_intermodal_graph:70 - Started downloading and parsing walk graph...
2024-09-23 16:40:15.062 | DEBUG    | iduedu.modules.intermodal_builder:get_intermodal_graph:77 - Started downloading and parsing public trasport graph...
2024-09-23 16:40:17.996 | DEBUG    | iduedu.modules.downloaders:get_routes_by_poly:113 - Downloading routes from OSM with type <tram> ...
2024-09-23 16:40:18.066 | DEBUG    | iduedu.modules.downloaders:get_routes_by_poly:113 - Downloading routes from OSM with type <bus> ...
2024-09-23 16:40:18.070 | DEBUG    | iduedu.modules.downloaders:get_routes_by_poly:113 - Downloading routes from OSM with type <subway> ...
2024-09-23 16:4

In [15]:
adj_drive = availability_matrix(g.graph, settlement_points,settlement_points,local_crs=local_crs)
adj_inter = availability_matrix(inter, settlement_points, settlement_points, local_crs=local_crs)

In [16]:
grader = AdvancedGrader(local_crs)
cri = grader.get_criteria(graded_terr=graded_terr,
                          points=settlement_points,
                          polygons=settlement_polygons,
                          r_stops=railway_stations,
                          b_stops=bus_stops,
                          aero=aero,
                          adj_mx_drive=adj_drive,
                          adj_mx_inter=adj_inter)


In [17]:
grader.interpret_gdf()

[('Белев',
  ['Территория расположена в непосредственной близости от одной из региональных трасс (в радиус 5 км от границ территории попадает хотя бы 1 узел региональной трассы, являющейся приоритетной, а ближайший федеральный узел находится в не более чем 100км);',
   'Территория попадает в IV квартиль связности (худшие 25% МО) на личном транспорте;',
   'Территория попадает в IV квартиль связности (худшие 25% МО) на общественном транспорте;',
   'В радиусе 15 км отсутствуют порты/причалы/переправы, аэродромы.'])]

## Indicators of the territory (graph-based)

In [18]:
# services
railway_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
railway_roads = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway.geojson')
bus_stops = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
bus_routes = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_bus_routes.geojson')
bus_routes['route'] = bus_routes['desc'] 
fuel_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия fuel.geojson')
local_aerodrome = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')
local_aerodrome['geometry'] = shapely.centroid(local_aerodrome['geometry']).set_crs(local_aerodrome.crs)
international_aerodrome = local_aerodrome[local_aerodrome['aerodrome:type']=='international']
international_aerodrome['geometry'] = shapely.centroid(international_aerodrome['geometry']).set_crs(international_aerodrome.crs)

# nature objects
water = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия water.geojson')
nature_reserve = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия nature_reserve.geojson')

capital =ox.geocode_to_gdf('N34389350',by_osmid=True)
districts_polygons = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/district.geojson')
settlement_polygons = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')
settlement_points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')



In [19]:
services = create_service_dict(
                                railway_stations=railway_stations,
                                railway_paths=railway_roads,
                                bus_stops=bus_stops,
                                bus_routes=bus_routes,
                                fuel_stations=fuel_stations,
                                local_aerodrome=local_aerodrome,
                                international_aerodrome=international_aerodrome,
                                water_objects=water,
                                nature_reserve=nature_reserve,          
                                local_crs=local_crs)

In [20]:
t = indicator_territory(g.graph,custom_terr,services,capital,admin_centers,settlement_points,districts_polygons,local_crs,)
t

,name,geometry,to_region_admin_center_km,to_reg_1_km,fuel_stations_accessibility_min,number_of_fuel_stations,local_aerodrome_accessibility_min,number_of_local_aerodrome,international_aerodrome_accessibility_min,number_of_international_aerodrome,...,number_of_bus_stops,number_of_water_objects,water_objects_accessibility_min,number_of_nature_reserve,nature_reserve_accessibility_min,train_path_length_km,number_of_bus_routes,to_nearest_district_center_km,to_nearest_settlement_km,road_density_km/km2
0,Белев,"POLYGON ((305406.670 5964767.649, 305378.241 5...",121.429,0.291,0.0,3.0,97.482,0.0,None,0,...,20.0,32,0.0,0,89.54,14.844,0,0.945,0.945,1.483


## Indicators of the area (graph-based)

In [21]:
#area polygons
country_polygon = ox.geocode_to_gdf('RUSSIA')
country_polygon['status'] = 'country'
country_polygon['layer'] = 'country'

In [23]:
import pandas as pd

In [57]:
con = get_adj_matrix_gdf_to_gdf(settlement_points.to_crs(local_crs),settlement_points.to_crs(local_crs),g.graph,'time_min')


/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/pandas/core/construction.py:820: RuntimeWarning: overflow encountered in cast
  subarr = np.asarray(arr, dtype=dtype)


In [46]:
def service_accessibility2(settlements_points, districts, services, local_crs):
    
    res = gpd.sjoin(settlements_points, districts, how="left", predicate="within")
    grouped_median = res.groupby('index_right').median(numeric_only=True)
    districts_with_index = districts[['name', 'layer','status','geometry']].to_crs(local_crs).reset_index()
    result = pd.merge(districts_with_index, grouped_median, left_on='index', right_on='index_right')
    for service in ['railway_stations', 'fuel_stations', 'ports', 'local_aerodrome', 'international_aerodrome', 'bus_stops']:
        if services[service].empty:
            result[f'{service}_accessibility_min'] = None
            result[f'number_of_{service}'] = 0
        else:
            joined = gpd.sjoin(services[service].to_crs(local_crs), districts, how="left", predicate='within')
            service_counts = joined.groupby('index_right').size().reset_index(name=f'number_of_{service}')
            result = result.merge(service_counts, how='left', left_on='index', right_on='index_right', suffixes=('', f'_{service}'))
        print(result.columns)       
        return result
    result = result.drop(columns=[col for col in result.columns if 'index_right' in col])
    numeric_cols = result.select_dtypes(include='number').columns
    result[numeric_cols] = result[numeric_cols].fillna(0)
    print(result.columns)
    column_order = [
    'name', 'layer','status', 'geometry',
    'number_of_railway_stations', 'railway_stations_accessbility_min',
    'number_of_fuel_stations', 'fuel_stations_accessbility_min',
    'number_of_ports', 'ports_accessibility_min',
    'number_of_local_aerodrome', 'local_aerodrome_accessbility_min',
    'number_of_international_aerodrome', 'international_aerodrome_accessibility_min',
    'number_of_bus_stops']

    result = result[column_order]       
    result = result.reset_index(drop=True)

    
    return result
service_accessibility2(settlement_points.to_crs(local_crs), districts_polygons.to_crs(local_crs),services,32636)

Index(['index', 'name', 'layer', 'status', 'geometry',
       'is_admin_centre_district', 'osm_id', '2019', '2020', '2021', '2022',
       '2023', 'admin_level', 'index_right', 'number_of_railway_stations'],
      dtype='object')


,index,name,layer,status,geometry,is_admin_centre_district,osm_id,2019,2020,2021,2022,2023,admin_level,index_right,number_of_railway_stations
0,0,Арсеньевский район,Арсеньевский муниципальный район,муниципальный район,"POLYGON ((716004.761 5952411.216, 716204.571 5...",0.0,-1748968.0,9570.0,9471.0,9457.0,9414.0,9214.0,6.0,NaN,NaN
1,1,Белевский район,Белевский муниципальный район,муниципальный район,"POLYGON ((690655.846 5970748.594, 690775.936 5...",0.0,-1748969.0,19256.0,18821.0,18555.0,19055.0,18707.0,6.0,NaN,NaN
2,2,Богородицкий район,Борогодицкий муниципальный район,муниципальный район,"POLYGON ((820635.295 5964987.176, 820812.522 5...",0.0,-1748970.0,50425.0,49942.0,49447.0,49782.0,49571.0,6.0,NaN,NaN
3,3,Веневский район,Веневский муниципальный район,муниципальный район,"POLYGON ((815882.425 6032567.814, 816000.612 6...",0.0,-1748971.0,31293.0,31105.0,30835.0,34203.0,34066.0,6.0,NaN,NaN
4,4,Воловский район,Воловский муниципальный район,муниципальный район,"POLYGON ((814388.883 5933869.093, 814985.427 5...",0.0,-1748972.0,13351.0,13343.0,13314.0,13944.0,13824.0,6.0,NaN,NaN
5,5,Алексин,городской округ Алексин,городской округ,"POLYGON ((749855.647 6043718.447, 750071.385 6...",0.0,-1077203.0,67288.0,66645.0,66243.0,71051.0,70069.0,6.0,NaN,NaN
6,6,Донской,городской округ Донской,городской округ,"POLYGON ((844184.792 5988305.226, 844263.034 5...",1.0,-1748973.0,63014.0,62621.0,61477.0,63625.0,63038.0,6.0,NaN,NaN
7,7,Ефремов,городской округ Ефремов,городской округ,"POLYGON ((822739.390 5891144.015, 822856.784 5...",0.0,-1748975.0,55336.0,54937.0,54200.0,57585.0,56838.0,6.0,NaN,NaN
8,8,Новогуровский,городской округ Новогуровский,городской округ,"POLYGON ((778959.001 6046409.366, 779142.608 6...",1.0,-1748983.0,3401.0,3403.0,3342.0,3623.0,3617.0,6.0,NaN,NaN
9,9,Новомосковск,городской округ Новомосковск,городской округ,"POLYGON ((833357.642 6005750.703, 833750.462 6...",0.0,-1741297.0,134932.0,133791.0,133021.0,132006.0,130673.0,6.0,NaN,NaN


In [20]:
ind_area = indicator_area(g.graph, [settlement_polygons, districts_polygons,country_polygon],settlement_points,services, capital,local_crs,adj_drive,adj_inter,con)

Processing areas:   0%|          | 0/3 [00:00<?, ?it/s]2024-09-20 15:10:08.380 | INFO     | transport_frames.indicators.indicator_area:indicator_area:77 - Calculating service accessibility
/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
2024-09-20 15:10:08.671 | INFO     | transport_frames.indicators.indicator_area:indicator_area:96 - Calculating distance to region admin center and federal roads
2024-09-20 15:10:09.256 | INFO     | transport_frames.indicators.indicator_area:indicator_area:101 - Calculating train path lengths
2024-09-20 15:10:10.167 | INFO     | transport_frames.indicators.indicator_area:indicator_area:110 - Calculating number of bus routes
2024-09-20 15:10:10.537 | INFO     | transport_frames.indicators.indicator_area:indicator_area:120 - Calculating reg_1 road lengths
2024-09-20 15:10:10.973 | INFO   

In [21]:
ind_area[0]


,name,layer,status,geometry,number_of_railway_stations,railway_stations_accessbility_min,number_of_fuel_stations,fuel_stations_accessbility_min,number_of_ports,ports_accessibility_min,...,international_aerodrome_accessibility_min,number_of_bus_stops,connectivity_drive_min,to_region_admin_center_km,to_reg_1_km,train_path_length_km,number_of_bus_routes,reg1_length_km,reg2_length_km,road_density_km/km2
0,Манаенское,Арсеньевский муниципальный район,сельское поселение,"POLYGON ((319712.393 5950903.092, 319906.056 5...",0.0,34.540,0.0,28.5650,0,None,...,None,0.0,116.95150,120.892,37.273,0.000,0,0.000,0.000,0.206
1,Астаповское,Арсеньевский муниципальный район,сельское поселение,"POLYGON ((332771.965 5945970.982, 334000.051 5...",0.0,36.962,1.0,15.6390,0,None,...,None,8.0,101.46100,113.216,37.469,0.000,0,0.000,0.000,0.214
2,Арсеньево,Арсеньевский муниципальный район,городское поселение,"POLYGON ((343651.582 5955815.722, 343761.146 5...",0.0,32.587,0.0,1.4500,0,None,...,None,0.0,94.09450,99.934,36.312,0.000,0,0.000,0.000,2.160
3,Левобережное,Белевский муниципальный район,сельское поселение,"POLYGON ((296002.715 5971313.180, 296090.881 5...",0.0,14.673,0.0,12.8435,0,None,...,None,33.0,121.80975,129.963,11.549,9.481,0,49.421,49.421,0.348
4,Правобережное,Белевский муниципальный район,сельское поселение,"POLYGON ((312040.687 5965643.207, 312045.055 5...",0.0,16.732,1.0,13.3245,0,None,...,None,3.0,114.68475,128.916,15.604,0.000,0,0.000,0.000,0.206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Яснополянское,Щекинский муниципальный район,сельское поселение,"POLYGON ((380503.409 5990059.646, 380547.820 5...",0.0,6.513,1.0,5.6560,0,None,...,None,20.0,67.01000,31.178,9.649,4.823,0,9.418,9.418,0.380
80,Ясногорск,Ясногорский муниципальный район,городское поселение,"POLYGON ((413254.265 6040445.319, 413960.125 6...",1.0,0.602,2.0,1.0010,0,None,...,None,40.0,83.19850,42.754,14.412,10.439,3,0.000,0.000,2.473
81,Теляковское,Ясногорский муниципальный район,сельское поселение,"POLYGON ((409727.955 6047091.910, 409971.823 6...",0.0,15.930,0.0,10.9700,0,None,...,None,21.0,94.48050,59.156,28.648,6.224,0,0.000,0.000,0.308
82,Ревякинское,Ясногорский муниципальный район,сельское поселение,"POLYGON ((401013.172 6038205.892, 401014.069 6...",1.0,7.012,1.0,10.1530,0,None,...,None,47.0,80.30850,44.840,19.011,54.686,3,16.718,16.718,0.320
